## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [1]:
import pandas as pd
import sqlite3
import prep_utils as pu 
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

### **Database Connection Setup**

In [2]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

In [3]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards)
df_new_players = pu.prepare_players(df_players,df_players_teams)
display(df_new_players)

Dropping Attribute lgID in Coaches...
Creating attribute coach playoffs win ratio...
Creating attribute num coach awards...
Removing players from Players without any single played game...


,bioID,pos,firstseason,lastseason,height,weight,college,collegeOther,birthDate,deathDate
1,abrossv01w,F,0,0,74.0,169,Connecticut,,1980-07-09,0000-00-00
4,adamsjo01w,C,0,0,75.0,180,New Mexico,,1981-05-24,0000-00-00
8,aguilel01w,G,0,0,67.0,165,George Washington,,1976-10-15,0000-00-00
9,ajavoma01w,G,0,0,68.0,160,Rutgers,,1986-05-07,0000-00-00
11,aldrima01w,G,0,0,71.0,153,UNC Charlotte,,1973-09-15,0000-00-00
...,...,...,...,...,...,...,...,...,...,...
886,zakalok01w,C,0,0,78.0,174,,,1977-12-18,0000-00-00
887,zarafr01w,G,0,0,70.0,146,,,1976-12-08,0000-00-00
888,zellosh01w,G,0,0,70.0,155,Pittsburgh,,1986-08-28,0000-00-00
891,zirkozu01w,G,0,0,69.0,145,,,1980-06-06,0000-00-00


### **Data Preparation**
We will preparate the data in each table, by cleaning & formatting it so that it can be easily used by the machine learning models afterwards.